#### Setup

In [ ]:
from specific import *

orig_figure_saver = figure_saver

### Specify the experiments to compare

In [ ]:
experiments = ("all", "no_temporal_shifts")
fmt_exp = [experiment_name_dict[exp] for exp in experiments]
assert len(experiments) == 2
model_name = f"comparison_{'__'.join(experiments)}"
figure_saver = orig_figure_saver(sub_directory=model_name)

### Load data

In [ ]:
experiment_data = load_experiment_data(
    experiments,
    ignore=(
        "endog_data",
        "exog_data",
        "filled_datasets",
        "masked_datasets",
        "land_mask",
    ),
)

### Check that the masks are aligned

In [ ]:
comp_masks = [experiment_data[experiment]["master_mask"] for experiment in experiments]
assert all(np.all(comp_masks[0] == comp_mask) for comp_mask in comp_masks[1:])
master_mask = experiment_data[experiments[0]]["master_mask"]

comp_data = [data["y_test"].values for data in experiment_data.values()]
assert all(np.all(comp_data[0] == cd) for cd in comp_data[1:])

### Predict BA

In [ ]:
val_data = experiment_data[experiments[0]]["y_test"].values
masked_val_data = get_mm_data(val_data, master_mask, "val")

predictions = {}
errors = {}  # GFED4 - prediction.

for experiment, data in experiment_data.items():
    print("Experiment:", experiment)

    data["model"].n_jobs = get_ncpus()
    with parallel_backend("threading", n_jobs=get_ncpus()):
        pred = data["model"].predict(data["X_test"])

    predictions[experiment] = get_mm_data(pred, master_mask, "val")
    errors[experiment] = get_mm_data(val_data - pred, master_mask, "val")

    print("R2:", r2_score(val_data, pred))
    print("MSE:", mean_squared_error(val_data, pred))

In [ ]:
stacked = np.hstack(
    (
        get_unmasked(predictions[experiments[1]]).reshape(-1, 1),
        get_unmasked(predictions[experiments[0]]).reshape(-1, 1),
    )
)

In [ ]:
_ = plt.hist(
    stacked, "auto", density=True, histtype="bar", stacked=True, label=["a", "b"]
)
_ = plt.yscale("log")
_ = plt.legend(loc="best")

In [ ]:
diffs = predictions[experiments[1]] - predictions[experiments[0]]

In [ ]:
error_mag_diff = np.abs(errors[experiments[1]]) - np.abs(errors[experiments[0]])

In [ ]:
error_mean_mag_diff = np.mean(np.abs(errors[experiments[1]]), axis=0) - np.mean(
    np.abs(errors[experiments[0]]), axis=0
)

In [ ]:
np.mean(error_mag_diff)

In [ ]:
plt.hist(get_unmasked(errors[experiments[0]]), bins=100)
plt.yscale("log")

In [ ]:
plt.hist(get_unmasked(errors[experiments[1]]), bins=100)
plt.yscale("log")

In [ ]:
plt.hist(get_unmasked(error_mag_diff), bins=100)
plt.yscale("log")

In [ ]:
# Plotting params.
figsize = (5.1, 2.8)
mpl.rcParams["figure.figsize"] = figsize
coast_linewidth = 0.3
date_str = "2010-01 to 2015-04"

In [ ]:
boundaries = [-1e-2, -1e-3, -1e-4, 0, 1e-4, 1e-3, 1e-2]

fig = cube_plotting(
    diffs,
    title=f"BA <{experiments[1]} - {experiments[0]}>\n{date_str}",
    boundaries=boundaries,
    cmap="brewer_RdYlBu_11",
    cmap_midpoint=0,
    cmap_symmetric=True,
    colorbar_kwargs={
        "label": "Burned Area Fraction",
        "format": "%0.1e",
    },
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(fig, f"ba_{model_name}", sub_directory="predictions")

In [ ]:
boundaries = [-1e-1, 0, 0.1 ** 0.5, 1, 1e1]

fig = cube_plotting(
    diffs / predictions[experiments[0]],
    title=f"BA <{experiments[1]} - {experiments[0]}> / {experiments[0]}\n{date_str}",
    boundaries=boundaries,
    cmap="brewer_RdYlBu_11",
    cmap_midpoint=0,
    cmap_symmetric=False,
    colorbar_kwargs={
        "label": "1",
        "format": "%0.1e",
    },
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(fig, f"rel_ba_{model_name}", sub_directory="predictions")

In [ ]:
abs_diffs = np.abs(diffs)
vmax = np.max(abs_diffs)
boundaries = [1e-5, 1e-4, 1e-3, 1e-2, vmax]

fig = cube_plotting(
    abs_diffs,
    title=f"BA <|{experiments[1]} - {experiments[0]}|>\n{date_str}",
    boundaries=boundaries,
    cmap="YlOrRd",
    colorbar_kwargs={
        "label": "Burned Area Fraction",
        "format": "%0.1e",
    },
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(fig, f"ba_mean_abs_{model_name}", sub_directory="predictions")

In [ ]:
boundaries = [0.1, 0.316, 1, 3.16, 10]

fig = cube_plotting(
    np.abs(diffs) / predictions[experiments[0]],
    title=f"BA <|{experiments[1]} - {experiments[0]}|> / {experiments[0]}\n{date_str}",
    boundaries=boundaries,
    cmap="YlOrBr",
    colorbar_kwargs={
        "label": "1",
        "format": "%0.1e",
    },
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(
    fig, f"rel_ba_mean_abs_{model_name}", sub_directory="predictions"
)

#### Comparison of the error between the experiments

In [ ]:
boundaries = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
extend = "both"

for experiment, error in errors.items():
    fig = cube_plotting(
        np.abs(error),
        title=f"BA <|Error({experiment})|>\n{date_str}",
        boundaries=boundaries,
        extend=extend,
        cmap="inferno",
        colorbar_kwargs={
            "label": "BA Fraction",
            "format": "%0.1e",
        },
        coastline_kwargs={"linewidth": coast_linewidth},
        log=True,
    )
    figure_saver.save_figure(
        fig, f"error_mag_{experiment}_{model_name}", sub_directory="predictions"
    )

In [ ]:
boundaries = [
    np.min(error_mag_diff),
    -1e-2,
    -1e-3,
    0,
    1e-3,
    1e-2,
    np.max(error_mag_diff),
]

fig = cube_plotting(
    error_mag_diff,
    #     title=f"BA <|Error({experiments[1]})| - |Error({experiments[0]})|>\n{date_str}",
    title="",
    boundaries=boundaries,
    cmap="brewer_RdYlBu_11",
    cmap_midpoint=0,
    cmap_symmetric=False,
    colorbar_kwargs={
        # "label": "BA Fraction",
        "label": f"|Err({fmt_exp[1]})| - |Err({fmt_exp[0]})|",
        "format": "%0.1e",
    },
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(
    fig, f"error_mag_diff_{model_name}", sub_directory="predictions"
)

In [ ]:
rel_error_mag_diff = np.mean(error_mag_diff, axis=0) / np.mean(masked_val_data, axis=0)
all_rel = get_unmasked(rel_error_mag_diff)

In [ ]:
plt.figure(figsize=(8, 4))
plt.ylabel("Counts")
plt.xlabel("Relative Error Magnitude Differences (+ve : all model improvement)")
_ = plt.hist(all_rel, log=True, bins=100)

In [ ]:
plt.figure(figsize=(8, 4))
plt.ylabel("Counts")
plt.xlabel("Relative Error Magnitude Differences (+ve : all model improvement)")
lim = 1
mask = (all_rel > -lim) & (all_rel < lim)
_ = plt.hist(all_rel[mask], log=True, bins=100)

In [ ]:
print(f"% >0: {100 * np.sum(all_rel > 0) / all_rel.size:0.1f}")
print(f"% <0: {100 * np.sum(all_rel < 0) / all_rel.size:0.1f}")

In [ ]:
cbar_fmt = ticker.FuncFormatter(lambda x, pos: simple_sci_format(x))

fig, cbar = cube_plotting(
    rel_error_mag_diff,
    #     title=f"BA <(|Error({experiments[1]})| - |Error({experiments[0]})|)> / <GFED4>\n{date_str}",
    title="",
    #     boundaries=boundaries,
    vmin=-0.5,
    vmax=0.5,
    nbins=2,
    cmap="PiYG",
    cmap_midpoint=0,
    cmap_symmetric=False,
    colorbar_kwargs={
        "label": f"<|Err({fmt_exp[1]})| - |Err({fmt_exp[0]})|> / <Ob.>",
        "format": cbar_fmt,
        "shrink": 0.5,
        "extendfrac": 0.1,
        "pad": 0.02,
    },
    coastline_kwargs={"linewidth": coast_linewidth},
    return_cbar=True,
)
cbar.ax.yaxis.label.set_size(7)
figure_saver.save_figure(
    fig, f"rel_error_mag_diff_{model_name}", sub_directory="predictions"
)